# Tree search

## Code

In [1]:
import typing
import math
import collections
import dataclasses
import abc
import itertools

T = typing.TypeVar("T", int, float)
Vector = typing.Tuple[T, T]
VectorComponent = typing.List[T]
Vertex = typing.List[typing.Tuple[T, T]]

def product_combination(iterable):
    first = iterable[0]

    for r in itertools.product(first, *iterable[1:]):
        yield r


def check_axis_intersect(that: Vector, z: T):
    x, y = that
    return y >= z >= x


def is_collision(that: Vertex, other: VectorComponent) -> bool:
    if len(that) != len(other):
        raise ValueError("Invalid arguments, different length of values")

    return all([check_axis_intersect(t, z) for t, z in zip(that, other)])


@dataclasses.dataclass
class VectorNode:
    data: VectorComponent
    value: T

    def __gt__(self, other):
        return self.value > other.value

    def __str__(self):
        return f"{self.data}, {self.value}"

    def __iter__(self):
        return iter([*self.data, self.value])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.vertex[item]

    def __setitem__(self, key, value):
        self.vertex[key] = value


def create_axis(_array: Vertex):
    all_data = []

    for arr in _array:
        x, y = arr
        d = math.fabs(y - x) / 2

        all_data.append(((x, x + d), (x + d, y)))

    return [*product_combination(all_data)]


@dataclasses.dataclass
class Node:
    vertex: Vertex
    nodes: list

    def append(self, node: VectorComponent):
        self.nodes.append(node)

    def is_collide(self, node: VectorComponent):
        return is_collision(self.vertex, node)

    def __len__(self):
        return len(self.nodes)

    def __iter__(self):
        return iter(self.nodes)


@dataclasses.dataclass
class NodeValue:
    vertex: Vertex
    node: VectorNode

    def __gt__(self, other: VectorNode):
        return self.value > other.value

    @property
    def value(self):
        return self.node.value


class NodeContainerInterface(abc.ABC):
    @property
    @abc.abstractmethod
    def axis(self) -> Vertex: ...

    @property
    @abc.abstractmethod
    def children(self): ...

    @property
    @abc.abstractmethod
    def node(self) -> Node: ...

    @property
    @abc.abstractmethod
    def is_parent(self) -> bool: ...

    @abc.abstractmethod
    def insert(self, verx: VectorComponent): ...

    @abc.abstractmethod
    def sort(self): ...

    @abc.abstractmethod
    def __iter__(self): ...


class NodeContainer:
    def __init__(self, data: Vertex, limit_divisions=1):
        self.__node = Node(vertex=data, nodes=[])
        self.__children = None
        self.__axis = data

        if limit_divisions > 0:
            self.__children = [
                NodeContainer(vertices, limit_divisions - 1) for vertices in create_axis(data)
            ]

    @property
    def axis(self):
        return [*self.__axis]

    @property
    def children(self) -> typing.List[NodeContainerInterface]:
        if self.__children:
            return [*self.__children]

        return []

    @property
    def node(self) -> Node:
        return self.__node

    @property
    def is_parent(self):
        return len(self.children) != 0

    def insert(self, verx: VectorComponent):
        return self._insert_loop(verx)

    def _insert_loop(self, verx: VectorComponent):
        children = collections.deque(self.children)

        while True:
            child = children.pop()
            node = child.node

            if node.is_collide(verx):
                if child.is_parent:
                    children = child.children

                    continue
                else:
                    node.append(verx)
                    return child

            if len(children) <= 0:
                return None

    def _insert_recursive(self, verx: VectorComponent):
        for child in self.children:
            node = child.node

            if node.is_collide(verx):
                if child.is_parent:
                    return child.insert(verx)
                else:
                    node.append(verx)
                    return child

        return None

    def sort(self):
        return self._get_iter_child_recursive()

    def __iter__(self):
        return iter(self._get_iter_child_recursive())

    def _get_iter_child_recursive(self):
        def get_iter_child(root, nodes=None):
            if nodes is None:
                nodes = []

            for child in root.children:
                node = child.node
                if child.is_parent:
                    get_iter_child(child, nodes)
                else:
                    if len(child.node) > 0:
                        nodes.append(child.node)

            return nodes

        return get_iter_child(self, [])

    def _get_iter_child_loop(self):
        pass


class TreeNode:
    def __init__(self):
        pass


## Initialize and config data

In [5]:
import sympy as sp
import numpy as np

N_COMPONENTS = 5

syms = sp.symbols(f'x1:{N_COMPONENTS +1 }')

ALL_SYMS = syms
LIMITS_RANGE = -3, 3

def get_n_syms(_range=(-1, 1)):
    return [_range for _ in range(len(syms))]

In [6]:
import operator
import functools

x1 = syms[0]
x2 = syms[1]
x3 = syms[2]

eq = sum(syms)
# eq = eq.subs(x1, 1/x1**2).subs(x2, x2**2).subs(x3, x3**3)
# eq = functools.reduce(operator.mul, syms, 1)**2 / 100

OBJ_FUNC = sp.lambdify(syms, eq, modules='numpy')

eq

x1 + x2 + x3 + x4 + x5

## Execution

In [7]:
import collections
import functools
import itertools

import pandas as pd

def tree_search_algorithm(f, axis, n_it=10, n_p=5, n_limit=0):
    tree = NodeContainer(axis, 2)

    all_it = collections.deque()
    all_it_route = collections.deque()

    for it in range(n_it):
        local_min_point = collections.deque()

        for _ in range(n_p):
            data = [np.random.uniform(*_) for _ in tree.axis]
            value = f(*data)

            tree.insert(VectorNode(data, value))

        for container in tree.sort():
            min_node = min(container)

            local_min_point.append(
                NodeValue(container.vertex, min_node)
            )

        if len(local_min_point) <= 0:
            continue

        local_min_node = min(local_min_point)

        all_it.append(list(min(local_min_point).node))
        all_it_route.append(local_min_node.vertex)

        tree = NodeContainer(local_min_node.vertex, 2)

    return all_it, all_it_route


all_it, all_it_route = tree_search_algorithm(
    OBJ_FUNC, [(-3, 3) for _ in range(len(syms))], 5, 5
)


In [8]:
pd.DataFrame(all_it_route)

,0,1,2,3,4
0,"(-3, -1.5)","(-1.5, 0.0)","(-3, -1.5)","(-3, -1.5)","(-3, -1.5)"
1,"(-2.625, -2.25)","(-0.375, 0.0)","(-2.625, -2.25)","(-3, -2.625)","(-2.25, -1.875)"
2,"(-2.4375, -2.34375)","(-0.1875, -0.09375)","(-2.34375, -2.25)","(-3, -2.90625)","(-2.15625, -2.0625)"
3,"(-2.4140625, -2.390625)","(-0.1875, -0.1640625)","(-2.34375, -2.3203125)","(-2.9765625, -2.953125)","(-2.15625, -2.1328125)"
4,"(-2.4140625, -2.408203125)","(-0.169921875, -0.1640625)","(-2.337890625, -2.33203125)","(-2.970703125, -2.96484375)","(-2.15625, -2.150390625)"


In [9]:
pd.DataFrame(all_it)

,0,1,2,3,4,5
0,-1.963201,-0.626843,-2.385887,-1.990638,-2.653493,-9.620062
1,-2.487142,-0.109375,-2.472420,-2.687822,-2.183217,-9.939975
2,-2.425159,-0.142742,-2.297132,-2.917220,-2.149879,-9.932132
3,-2.394344,-0.177474,-2.338817,-2.973699,-2.151891,-10.036225
4,-2.412075,-0.164972,-2.336516,-2.967662,-2.153628,-10.034854
